<a href="https://colab.research.google.com/github/sandeepgundeboina/LearningSpark/blob/main/SparkDeltaBucketing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install delta-spark==2.0.0
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("SparkDeltaBucketing") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.0.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()
from delta.tables import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
spark

In [ ]:
spark.conf.get('spark.sql.sources.bucketing.enabled')

'true'

In [ ]:
from pyspark.sql.functions import col, rand
df=spark.range(1,10000,1,10).select(col('id').alias('PK'),rand(10).alias('Attribute'))
df.show()

+---+-------------------+
| PK|          Attribute|
+---+-------------------+
|  1| 0.1709497137955568|
|  2| 0.8051143958005459|
|  3| 0.5775925576589018|
|  4| 0.9476047869880925|
|  5|    0.2093704977577|
|  6|0.36664222617947817|
|  7| 0.8078688178371882|
|  8| 0.7135143433452461|
|  9| 0.7195325566306053|
| 10|0.31335292311175456|
| 11| 0.8062503712025726|
| 12|0.10814914646176654|
| 13| 0.3362232980701172|
| 14| 0.8133304803837667|
| 15|0.47649428738170896|
| 16|  0.524728096293865|
| 17| 0.9701253460019921|
| 18| 0.6232167713919952|
| 19| 0.5089687568245219|
| 20| 0.5467504094508642|
+---+-------------------+
only showing top 20 rows



In [ ]:
df.count()

9999

In [ ]:
df.write.format('parquet').saveAsTable('non_bucketed_table')

In [ ]:
df.rdd.getNumPartitions()

10

In [ ]:
df.write.format('parquet').bucketBy(10,'PK').saveAsTable('bucketed_table')

In [ ]:
df1=spark.table('bucketed_table')
df2=spark.table('bucketed_table')

df3=spark.table('non_bucketed_table')
df4=spark.table('non_bucketed_table')

In [ ]:
df3.join(df4,'PK','inner').explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [PK#206L, Attribute#207, Attribute#213]
   +- BroadcastHashJoin [PK#206L], [PK#212L], Inner, BuildRight, false
      :- Filter isnotnull(PK#206L)
      :  +- FileScan parquet default.non_bucketed_table[PK#206L,Attribute#207] Batched: true, DataFilters: [isnotnull(PK#206L)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/content/spark-warehouse/non_bucketed_table], PartitionFilters: [], PushedFilters: [IsNotNull(PK)], ReadSchema: struct<PK:bigint,Attribute:double>
      +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [plan_id=125]
         +- Filter isnotnull(PK#212L)
            +- FileScan parquet default.non_bucketed_table[PK#212L,Attribute#213] Batched: true, DataFilters: [isnotnull(PK#212L)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/content/spark-warehouse/non_bucketed_table], PartitionFilters: [], PushedFilters: [IsNotNull(PK)], ReadSchema: stru

In [ ]:
spark.conf.set('spark.sql.autoBroadcastJoinThreshold',-1)
spark.conf.set('spark.sql.adaptive.enabled',False)

In [ ]:
df3.join(df4,'PK','inner').explain(mode='formatted')

== Physical Plan ==
* Project (9)
+- * SortMergeJoin Inner (8)
   :- * Sort (5)
   :  +- Exchange (4)
   :     +- * Filter (3)
   :        +- * ColumnarToRow (2)
   :           +- Scan parquet default.non_bucketed_table (1)
   +- * Sort (7)
      +- ReusedExchange (6)


(1) Scan parquet default.non_bucketed_table
Output [2]: [PK#206L, Attribute#207]
Batched: true
Location: InMemoryFileIndex [file:/content/spark-warehouse/non_bucketed_table]
PushedFilters: [IsNotNull(PK)]
ReadSchema: struct<PK:bigint,Attribute:double>

(2) ColumnarToRow [codegen id : 1]
Input [2]: [PK#206L, Attribute#207]

(3) Filter [codegen id : 1]
Input [2]: [PK#206L, Attribute#207]
Condition : isnotnull(PK#206L)

(4) Exchange
Input [2]: [PK#206L, Attribute#207]
Arguments: hashpartitioning(PK#206L, 200), ENSURE_REQUIREMENTS, [plan_id=261]

(5) Sort [codegen id : 2]
Input [2]: [PK#206L, Attribute#207]
Arguments: [PK#206L ASC NULLS FIRST], false, 0

(6) ReusedExchange [Reuses operator id: 4]
Output [2]: [PK#222L, Attri

In [ ]:
df2.join(df3,'PK','inner').explain(mode='formatted')

== Physical Plan ==
* Project (12)
+- * SortMergeJoin Inner (11)
   :- * Sort (5)
   :  +- Exchange (4)
   :     +- * Filter (3)
   :        +- * ColumnarToRow (2)
   :           +- Scan parquet default.bucketed_table (1)
   +- * Sort (10)
      +- Exchange (9)
         +- * Filter (8)
            +- * ColumnarToRow (7)
               +- Scan parquet default.non_bucketed_table (6)


(1) Scan parquet default.bucketed_table
Output [2]: [PK#194L, Attribute#195]
Batched: true
Location: InMemoryFileIndex [file:/content/spark-warehouse/bucketed_table]
PushedFilters: [IsNotNull(PK)]
ReadSchema: struct<PK:bigint,Attribute:double>

(2) ColumnarToRow [codegen id : 1]
Input [2]: [PK#194L, Attribute#195]

(3) Filter [codegen id : 1]
Input [2]: [PK#194L, Attribute#195]
Condition : isnotnull(PK#194L)

(4) Exchange
Input [2]: [PK#194L, Attribute#195]
Arguments: hashpartitioning(PK#194L, 200), ENSURE_REQUIREMENTS, [plan_id=354]

(5) Sort [codegen id : 2]
Input [2]: [PK#194L, Attribute#195]
Arguments: 

In [ ]:
df2.join(df1,'PK','inner').explain(mode='formatted')

== Physical Plan ==
* Project (10)
+- * SortMergeJoin Inner (9)
   :- * Sort (4)
   :  +- * Filter (3)
   :     +- * ColumnarToRow (2)
   :        +- Scan parquet default.bucketed_table (1)
   +- * Sort (8)
      +- * Filter (7)
         +- * ColumnarToRow (6)
            +- Scan parquet default.bucketed_table (5)


(1) Scan parquet default.bucketed_table
Output [2]: [PK#194L, Attribute#195]
Batched: true
Location: InMemoryFileIndex [file:/content/spark-warehouse/bucketed_table]
PushedFilters: [IsNotNull(PK)]
ReadSchema: struct<PK:bigint,Attribute:double>
SelectedBucketsCount: 10 out of 10

(2) ColumnarToRow [codegen id : 1]
Input [2]: [PK#194L, Attribute#195]

(3) Filter [codegen id : 1]
Input [2]: [PK#194L, Attribute#195]
Condition : isnotnull(PK#194L)

(4) Sort [codegen id : 1]
Input [2]: [PK#194L, Attribute#195]
Arguments: [PK#194L ASC NULLS FIRST], false, 0

(5) Scan parquet default.bucketed_table
Output [2]: [PK#230L, Attribute#231]
Batched: true
Location: InMemoryFileIndex [file

#####**END OF CODE**